# 공격성 규범 점수

## 평가내역에 따른 공격성 규범 산출

In [1]:
import sys
print(sys.executable)   # 예: C:\ProgramData\anaconda3\python.exe
print(sys.path)         # 패키지 탐색 경로(여기에 site-packages가 있어야 import 가능)

c:\ProgramData\anaconda3\python.exe
['c:\\ProgramData\\anaconda3\\python313.zip', 'c:\\ProgramData\\anaconda3\\DLLs', 'c:\\ProgramData\\anaconda3\\Lib', 'c:\\ProgramData\\anaconda3', '', 'C:\\Users\\USER\\AppData\\Roaming\\Python\\Python313\\site-packages', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\Pythonwin']


In [41]:
import awswrangler as wr
query_df = wr.athena.read_sql_query(
    """
    select a.school_code, a.school_name, b.school_grade, b.school_num, b.class_name, c.psy_name, c.psy_code, c.close_yn, f.student_num, d.target_code, e.user_testing_no, e.question_no, e.student_code 
    from school_info a
    inner join school_class b on a.school_code = b.school_code
    inner join psy_class c on b.class_code = c.class_code
    inner join psy_target d on c.class_code = d.class_code and c.psy_code = d.psy_code
    inner join at_user_testing_paper_pn e on d.user_testing_no = e.user_testing_no
    inner join school_student f on d.target_code = f.student_code and d.class_code = f.class_code
    where c.close_yn = 'Y' and e.question_no in (4, 5, 6, 7, 14)
    """
    , database="schoolfriends"
)

In [42]:
query_df

,school_code,school_name,school_grade,school_num,class_name,psy_name,psy_code,close_yn,student_num,target_code,user_testing_no,question_no,student_code
0,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",4,AC000120256042D2E003
1,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",5,AC000120256042D2E003
2,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E003
3,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E009
4,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,PERSONAL,PERSONAL,"""3""","""1""",학지초등학교3학년1반,"""25년 7월 2회차_연구개발검수""",P202507046A44,Y,<NA>,M1000000120253017C9D004,b06c46be00ca45afba53,6,M1000000120253017C9D001
8490,PERSONAL,PERSONAL,"""3""","""1""",학지초등학교3학년1반,"""25년 7월 2회차_연구개발검수""",P202507046A44,Y,<NA>,M1000000120253017C9D004,b06c46be00ca45afba53,6,M1000000120253017C9D002
8491,PERSONAL,PERSONAL,"""3""","""1""",학지초등학교3학년1반,"""25년 7월 2회차_연구개발검수""",P202507046A44,Y,<NA>,M1000000120253017C9D004,b06c46be00ca45afba53,7,M1000000120253017C9D002
8492,PERSONAL,PERSONAL,"""3""","""1""",학지초등학교3학년1반,"""25년 7월 2회차_연구개발검수""",P202507046A44,Y,<NA>,M1000000120253017C9D004,b06c46be00ca45afba53,7,M1000000120253017C9D003


In [43]:
query_df.columns

Index(['school_code', 'school_name', 'school_grade', 'school_num',
       'class_name', 'psy_name', 'psy_code', 'close_yn', 'student_num',
       'target_code', 'user_testing_no', 'question_no', 'student_code'],
      dtype='object')

In [44]:
# 지명수 구하는 함수
def count_point(df, student_list, item_no) :
    import pandas as pd
    if df['psy_code'].nunique() > 1 :
        raise ValueError('psy_code 컬럼에 여러 값이 존재합니다.')
    else :
        temp_df = df[df['question_no'].isin(item_no)] # 해당 문항 필터링
        count_list = []
        for student in student_list :
            if len(temp_df[temp_df['student_code']==student]) >= 1 :
                freq = len(temp_df[temp_df['student_code']==student])
            elif len(temp_df[temp_df['student_code']==student]) == 0 :
                freq = 0
            count_list.append(freq)
        count_point_df = pd.DataFrame({'student_code': student_list, 'num_point': count_list})
        return count_point_df, df['psy_code'].iloc[0]

# 학생 리스트 구하는 함수
def student_list(df) :
    student_list = df['target_code'].unique().tolist()
    return student_list, df['psy_code'].iloc[0]

In [47]:
df = query_df[query_df['psy_name'].str.contains('이아린비활')]
print(df)

    school_code school_name school_grade school_num class_name  \
756      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
757      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
758      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
759      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
760      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
..          ...         ...          ...        ...        ...   
946      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
947      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
948      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
949      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
950      AC0001      학지초등학교          "5"        "1"    "5학년1반"   

               psy_name       psy_code close_yn  student_num  \
756  "25년 7월 2회차_이아린비활"  P202507183B0A        Y         <NA>   
757  "25년 7월 2회차_이아린비활"  P202507183B0A        Y         <NA>   
758  "25년 7월 2회

In [6]:
#! 디버깅용으로 데이터 엑셀로 저장
import pandas as pd
df.to_excel(f'scoring_etl_check_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.xlsx', index=False)

In [16]:
student_list_obj, psy_code = student_list(df)
print(student_list_obj, psy_code)
print(len(student_list_obj))

['AC000120255019FA1022', 'AC000120255019FA1021', 'AC000120255019FA1020', 'AC000120255019FA1019', 'AC000120255019FA1018', 'AC000120255019FA1017', 'AC000120255019FA1016', 'AC000120255019FA1014', 'AC000120255019FA1013', 'AC000120255019FA1012', 'AC000120255019FA1011', 'AC000120255019FA1010', 'AC000120255019FA1009', 'AC000120255019FA1008', 'AC000120255019FA1007', 'AC000120255019FA1006', 'AC000120255019FA1005', 'AC000120255019FA1004', 'AC000120255019FA1003', 'AC000120255019FA1002', 'AC000120255019FA1001'] P202507183B0A
21


In [17]:
aggression_point_df, psy_code = count_point(df, student_list_obj, [4,5,6,7])
n14_point_df, psy_code = count_point(df, student_list_obj, [14])
print(aggression_point_df, psy_code)
print(n14_point_df, psy_code)

            student_code  num_point
0   AC000120255019FA1022          4
1   AC000120255019FA1021          8
2   AC000120255019FA1020          4
3   AC000120255019FA1019          8
4   AC000120255019FA1018          9
5   AC000120255019FA1017          9
6   AC000120255019FA1016          2
7   AC000120255019FA1014          8
8   AC000120255019FA1013          4
9   AC000120255019FA1012          9
10  AC000120255019FA1011          7
11  AC000120255019FA1010          3
12  AC000120255019FA1009          3
13  AC000120255019FA1008          7
14  AC000120255019FA1007          5
15  AC000120255019FA1006         10
16  AC000120255019FA1005          9
17  AC000120255019FA1004          1
18  AC000120255019FA1003          7
19  AC000120255019FA1002          3
20  AC000120255019FA1001         11 P202507183B0A
            student_code  num_point
0   AC000120255019FA1022          1
1   AC000120255019FA1021          1
2   AC000120255019FA1020          1
3   AC000120255019FA1019          0
4   AC00012025

In [45]:
def aggression_norms(df, celling_parameter=67) :
    import pandas as pd
    student_list_obj, psy_code = student_list(df)
    aggression_point_df, psy_code = count_point(df, student_list_obj, [4,5,6,7])
    n14_point_df, psy_code = count_point(df, student_list_obj, [14])

    aggression_norms_df = pd.merge(aggression_point_df, n14_point_df, on='student_code', suffixes=('_agg', '_14no'))
    aggression_norms_df['agg*14no_num_point'] = aggression_norms_df['num_point_agg'] * aggression_norms_df['num_point_14no']
    aggression_norms_df.sort_values(by='agg*14no_num_point', ascending=False, inplace=True)
    
    top5_aggression_norms_df = aggression_norms_df.head(5)
    top5_mean = top5_aggression_norms_df['agg*14no_num_point'].mean()
    total_mean = aggression_norms_df['agg*14no_num_point'].mean()
    weighted_mean = 0.4 * (top5_mean) + 0.6 * (total_mean)
    weighted_mean = 4 * ((min(max(weighted_mean, 0), celling_parameter)-0)/celling_parameter)+1
    result_df = pd.DataFrame({'psy_code': [psy_code], 'top5_mean': [top5_mean], 'total_mean': [total_mean], 'weighted_mean': [weighted_mean]})
    result_df['reg_date'] = pd.Timestamp.now()
    result_df['purpose'] = f'"purpose=calibration;celling_parameter={celling_parameter};author=Yumin"'
    return result_df

In [48]:
result_df = aggression_norms(df)
print(result_df)

        psy_code  top5_mean  total_mean  weighted_mean  \
0  P202507183B0A       27.8   11.571429        2.07838   

                    reg_date  \
0 2025-09-30 17:35:28.158909   

                                             purpose  
0  "purpose=calibration;celling_parameter=67;auth...  


## raw 쿼리 결과 -> 그룹별 채점 결과 변환

In [52]:
import awswrangler as wr
query_df = wr.athena.read_sql_query(
    """
    select a.school_code, a.school_name, b.school_grade, b.school_num, b.class_name, c.psy_name, c.psy_code, c.close_yn, f.student_num, d.target_code, e.user_testing_no, e.question_no, e.student_code 
    from school_info a
    inner join school_class b on a.school_code = b.school_code
    inner join psy_class c on b.class_code = c.class_code
    inner join psy_target d on c.class_code = d.class_code and c.psy_code = d.psy_code
    inner join at_user_testing_paper_pn e on d.user_testing_no = e.user_testing_no
    inner join school_student f on d.target_code = f.student_code and d.class_code = f.class_code
    where c.close_yn = 'Y' and e.question_no in (4,5,6,7,14)
    """
    , database="schoolfriends")

In [53]:
query_df

,school_code,school_name,school_grade,school_num,class_name,psy_name,psy_code,close_yn,student_num,target_code,user_testing_no,question_no,student_code
0,PERSONAL,PERSONAL,"""6""","""1""","""6학년2반""","""25년 7월 1회차""",P20250711FD39,Y,<NA>,M1000005520256019C95008,ec497b0eb6ae49368fe5,6,M1000005520256019C95007
1,PERSONAL,PERSONAL,"""6""","""1""","""6학년2반""","""25년 7월 1회차""",P20250711FD39,Y,<NA>,M1000005520256019C95008,ec497b0eb6ae49368fe5,6,M1000005520256019C95009
2,PERSONAL,PERSONAL,"""6""","""1""","""6학년2반""","""25년 7월 1회차""",P20250711FD39,Y,<NA>,M1000005520256019C95008,ec497b0eb6ae49368fe5,7,M1000005520256019C95004
3,PERSONAL,PERSONAL,"""6""","""1""","""6학년2반""","""25년 7월 1회차""",P20250711FD39,Y,<NA>,M1000005520256019C95008,ec497b0eb6ae49368fe5,7,M1000005520256019C95005
4,PERSONAL,PERSONAL,"""6""","""1""","""6학년2반""","""25년 7월 1회차""",P20250711FD39,Y,<NA>,M1000005520256019C95008,ec497b0eb6ae49368fe5,7,M1000005520256019C95010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,PERSONAL,PERSONAL,"""6""","""1""","""6학년2반""","""25년 7월 1회차""",P20250711FD39,Y,<NA>,M1000005520256019C95008,ec497b0eb6ae49368fe5,4,M1000005520256019C95007
8490,PERSONAL,PERSONAL,"""6""","""1""","""6학년2반""","""25년 7월 1회차""",P20250711FD39,Y,<NA>,M1000005520256019C95008,ec497b0eb6ae49368fe5,4,M1000005520256019C95009
8491,PERSONAL,PERSONAL,"""6""","""1""","""6학년2반""","""25년 7월 1회차""",P20250711FD39,Y,<NA>,M1000005520256019C95008,ec497b0eb6ae49368fe5,5,M1000005520256019C95006
8492,PERSONAL,PERSONAL,"""6""","""1""","""6학년2반""","""25년 7월 1회차""",P20250711FD39,Y,<NA>,M1000005520256019C95008,ec497b0eb6ae49368fe5,5,M1000005520256019C95007


In [54]:
import pandas as pd

psy_code_grouped = query_df.groupby('psy_code')
result_df = pd.DataFrame()
for psy_code, group in psy_code_grouped :
    result_df = pd.concat([result_df, aggression_norms(group)])

In [55]:
result_df

,psy_code,top5_mean,total_mean,weighted_mean,reg_date,purpose
0,P202507041AFF,17.200000,17.200000,2.026866,2025-09-30 17:38:27.416093,"""purpose=calibration;celling_parameter=67;auth..."
0,P202507046A44,8.400000,8.400000,1.501493,2025-09-30 17:38:27.422828,"""purpose=calibration;celling_parameter=67;auth..."
0,P202507075FAD,3.500000,3.500000,1.208955,2025-09-30 17:38:27.429166,"""purpose=calibration;celling_parameter=67;auth..."
0,P202507086502,6.000000,6.000000,1.358209,2025-09-30 17:38:27.434944,"""purpose=calibration;celling_parameter=67;auth..."
0,P20250708740B,70.000000,36.200000,3.968358,2025-09-30 17:38:27.462677,"""purpose=calibration;celling_parameter=67;auth..."
0,P2025070965A7,2.000000,2.000000,1.119403,2025-09-30 17:38:27.467758,"""purpose=calibration;celling_parameter=67;auth..."
0,P202507098064,2.333333,2.333333,1.139303,2025-09-30 17:38:27.472969,"""purpose=calibration;celling_parameter=67;auth..."
0,P202507118C5B,2.000000,2.000000,1.119403,2025-09-30 17:38:27.478356,"""purpose=calibration;celling_parameter=67;auth..."
0,P20250711FD39,50.800000,27.800000,3.208955,2025-09-30 17:38:27.489476,"""purpose=calibration;celling_parameter=67;auth..."
0,P202507154E91,186.400000,59.782609,5.000000,2025-09-30 17:38:27.513235,"""purpose=calibration;celling_parameter=67;auth..."


In [18]:
# 언제 등록하고 목적이 무엇인지 같이 기록
result_df['reg_date'] = pd.Timestamp.now()
result_df['purpose'] = '"purpose=calibration;celling_parameter=67;author=Yumin"'

In [19]:
# 결과를 S3에 저장
wr.s3.to_csv(result_df, f's3://schoolfriends-bym/result/aggression_norms/aggression_norms_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.csv', index=False)

{'paths': ['s3://schoolfriends-bym/result/aggression_norms/aggression_norms_20250908_094510.csv'],
 'partitions_values': {}}

In [17]:
# s3_folder_macro 소스에서 크롤러 생성해서 테이블화

## celling_parameter range로 조정하여 탐색

In [70]:
def search_celling_aggression_norms(df, celling_parameter) :
    import pandas as pd
    student_list_obj, psy_code = student_list(df)
    aggression_point_df, psy_code = count_point(df, student_list_obj, [4,5,6,7])
    n14_point_df, psy_code = count_point(df, student_list_obj, [14])

    aggression_norms_df = pd.merge(aggression_point_df, n14_point_df, on='student_code', suffixes=('_agg', '_14no'))
    aggression_norms_df['agg*14no_num_point'] = aggression_norms_df['num_point_agg'] * aggression_norms_df['num_point_14no']
    aggression_norms_df.sort_values(by='agg*14no_num_point', ascending=False, inplace=True)
    
    top5_aggression_norms_df = aggression_norms_df.head(5)
    top5_mean = top5_aggression_norms_df['agg*14no_num_point'].mean()
    total_mean = aggression_norms_df['agg*14no_num_point'].mean()
    weighted_mean = 0.4 * (top5_mean) + 0.6 * (total_mean)
    weighted_mean_Normalization = 4 * ((min(max(weighted_mean, 0), celling_parameter)-0)/celling_parameter)+1
    result_df = pd.DataFrame({'psy_code': [psy_code], 'top5_mean': [top5_mean], 'total_mean': [total_mean], 'weighted_mean': [weighted_mean], 'weighted_mean_Normalization': [weighted_mean_Normalization]})
    result_df['reg_date'] = pd.Timestamp.now()
    result_df['purpose'] = f'"purpose=calibration;celling_parameter={celling_parameter};author=Yumin"'
    return result_df

In [71]:
import awswrangler as wr
query_df = wr.athena.read_sql_query(
    """
    select a.school_code, a.school_name, b.school_grade, b.school_num, b.class_name, c.psy_name, c.psy_code, c.close_yn, f.student_num, d.target_code, e.user_testing_no, e.question_no, e.student_code 
    from school_info a
    inner join school_class b on a.school_code = b.school_code
    inner join psy_class c on b.class_code = c.class_code
    inner join psy_target d on c.class_code = d.class_code and c.psy_code = d.psy_code
    inner join at_user_testing_paper_pn e on d.user_testing_no = e.user_testing_no
    inner join school_student f on d.target_code = f.student_code and d.class_code = f.class_code
    where c.close_yn = 'Y' and e.question_no in (4,5,6,7,14)
    """
    , database="schoolfriends")

In [72]:
query_df

,school_code,school_name,school_grade,school_num,class_name,psy_name,psy_code,close_yn,student_num,target_code,user_testing_no,question_no,student_code
0,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",4,AC000120256042D2E003
1,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",5,AC000120256042D2E003
2,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E003
3,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E009
4,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,6,""""""
8490,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,7,""""""
8491,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567012
8492,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567021


In [73]:
import pandas as pd

psy_code_grouped = query_df.groupby('psy_code')
result_df = pd.DataFrame()
for parameter in range(30, 100) :
    print(f'celling_parameter: {parameter}')
    for psy_code, group in psy_code_grouped :
        result_df = pd.concat([result_df, search_celling_aggression_norms(group, celling_parameter=parameter)])

celling_parameter: 30
celling_parameter: 31
celling_parameter: 32
celling_parameter: 33
celling_parameter: 34
celling_parameter: 35
celling_parameter: 36
celling_parameter: 37
celling_parameter: 38
celling_parameter: 39
celling_parameter: 40
celling_parameter: 41
celling_parameter: 42
celling_parameter: 43
celling_parameter: 44
celling_parameter: 45
celling_parameter: 46
celling_parameter: 47
celling_parameter: 48
celling_parameter: 49
celling_parameter: 50
celling_parameter: 51
celling_parameter: 52
celling_parameter: 53
celling_parameter: 54
celling_parameter: 55
celling_parameter: 56
celling_parameter: 57
celling_parameter: 58
celling_parameter: 59
celling_parameter: 60
celling_parameter: 61
celling_parameter: 62
celling_parameter: 63
celling_parameter: 64
celling_parameter: 65
celling_parameter: 66
celling_parameter: 67
celling_parameter: 68
celling_parameter: 69
celling_parameter: 70
celling_parameter: 71
celling_parameter: 72
celling_parameter: 73
celling_parameter: 74
celling_pa

In [74]:
print(result_df)

         psy_code  top5_mean  total_mean  weighted_mean  \
0   P202507041AFF       17.2   17.200000      17.200000   
0   P202507046A44        8.4    8.400000       8.400000   
0   P202507075FAD        3.5    3.500000       3.500000   
0   P202507086502        6.0    6.000000       6.000000   
0   P20250708740B       70.0   36.200000      49.720000   
..            ...        ...         ...            ...   
0   P20250926990C       14.4    5.333333       8.960000   
0   P20250926E621       85.2   29.440000      51.744000   
0   P20250926F99B       12.2    6.400000       8.720000   
0   P202509297849       48.4   18.391304      30.394783   
0   P20250929B2C3      109.2   32.285714      63.051429   

    weighted_mean_Normalization                   reg_date  \
0                      3.293333 2025-10-01 14:10:18.242565   
0                      2.120000 2025-10-01 14:10:18.249522   
0                      1.466667 2025-10-01 14:10:18.256045   
0                      1.800000 2025-10-01 

In [75]:
# s3에 업로드
wr.s3.to_csv(result_df, f's3://schoolfriends-bym/result/aggression_norms/aggression_norms_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.csv', index=False)

{'paths': ['s3://schoolfriends-bym/result/aggression_norms/aggression_norms_20251001_141111.csv'],
 'partitions_values': {}}

## 평가마다 학생의 공격 지명 * 친구 지명

In [76]:
# 지명수 구하는 함수
def count_point(df, student_list, item_no) :
    import pandas as pd
    if df['psy_code'].nunique() > 1 :
        raise ValueError('psy_code 컬럼에 여러 값이 존재합니다.')
    else :
        temp_df = df[df['question_no'].isin(item_no)] # 해당 문항 필터링
        count_list = []
        for student in student_list :
            if len(temp_df[temp_df['student_code']==student]) >= 1 :
                freq = len(temp_df[temp_df['student_code']==student])
            elif len(temp_df[temp_df['student_code']==student]) == 0 :
                freq = 0
            count_list.append(freq)
        count_point_df = pd.DataFrame({'student_code': student_list, 'num_point': count_list})
        return count_point_df, df['psy_code'].iloc[0]
    
# 학생 리스트 구하는 함수
def student_list(df) :
    student_list = df['target_code'].unique().tolist()
    return student_list, df['psy_code'].iloc[0]

In [77]:
query_df

,school_code,school_name,school_grade,school_num,class_name,psy_name,psy_code,close_yn,student_num,target_code,user_testing_no,question_no,student_code
0,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",4,AC000120256042D2E003
1,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",5,AC000120256042D2E003
2,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E003
3,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E009
4,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,6,""""""
8490,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,7,""""""
8491,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567012
8492,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567021


In [78]:
import pandas as pd

psy_code_grouped = query_df.groupby('psy_code')
result_df = pd.DataFrame()
for psy_code, group in psy_code_grouped :
    student_list_obj, psy_code = student_list(group)
    aggression_point_df, psy_code = count_point(group, student_list_obj, [4,5,6,7])
    n14_point_df, psy_code = count_point(group, student_list_obj, [14])
    merged_df = pd.merge(aggression_point_df, n14_point_df, on='student_code', suffixes=('_agg', '_14no'))
    merged_df['agg*14no_num_point'] = merged_df['num_point_agg'] * merged_df['num_point_14no']
    merged_df['psy_code'] = psy_code
    print(merged_df)
    result_df = pd.concat([result_df, merged_df])

              student_code  num_point_agg  num_point_14no  agg*14no_num_point  \
0  M1000000120253017C9D005             10               2                  20   
1  M1000000120253017C9D004             12               4                  48   
2  M1000000120253017C9D003              5               3                  15   
3  M1000000120253017C9D002              5               0                   0   
4  M1000000120253017C9D001              3               1                   3   

        psy_code  
0  P202507041AFF  
1  P202507041AFF  
2  P202507041AFF  
3  P202507041AFF  
4  P202507041AFF  
              student_code  num_point_agg  num_point_14no  agg*14no_num_point  \
0  M1000000120253017C9D005              7               2                  14   
1  M1000000120253017C9D004              7               2                  14   
2  M1000000120253017C9D003              7               2                  14   
3  M1000000120253017C9D002              8               0                  

In [79]:
print(result_df)

               student_code  num_point_agg  num_point_14no  \
0   M1000000120253017C9D005             10               2   
1   M1000000120253017C9D004             12               4   
2   M1000000120253017C9D003              5               3   
3   M1000000120253017C9D002              5               0   
4   M1000000120253017C9D001              3               1   
..                      ...            ...             ...   
16     DC03142025602FCDA005              2               2   
17     DC03142025602FCDA004              4               8   
18     DC03142025602FCDA003             17               7   
19     DC03142025602FCDA002              3               3   
20     DC03142025602FCDA001              0               2   

    agg*14no_num_point       psy_code  
0                   20  P202507041AFF  
1                   48  P202507041AFF  
2                   15  P202507041AFF  
3                    0  P202507041AFF  
4                    3  P202507041AFF  
..             

In [80]:
# s3에 업로드
wr.s3.to_csv(result_df, f's3://schoolfriends-bym/result/aggression_point/aggression_point_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.csv', index=False)

{'paths': ['s3://schoolfriends-bym/result/aggression_point/aggression_point_20251001_141131.csv'],
 'partitions_values': {}}

In [83]:
# s3_folder_macro 소스에서 크롤러 생성해서 테이블화
# 특정 크롤러 생성 및 실행
import boto3
# s3 폴더 별로 크롤러 만들어서 아테네 테이블 만들때
glue = boto3.client("glue", region_name="ap-northeast-2")

# 공통 파라미터
role_name = 'arn:aws:iam::932744610695:role/service-role/AWSGlueServiceRole-Schoolfriends'
database_name = 'schoolfriends'
bucket_name = 'schoolfriends-bym'

crawler_name = "crawler_aggression_point"
s3_target_path = f"s3://schoolfriends-bym/result/aggression_point/"

try:
    response = glue.create_crawler(
        Name=crawler_name,
        Role=role_name,
        DatabaseName=database_name,
        Targets={'S3Targets': [{'Path': s3_target_path}]},
        SchemaChangePolicy={
            'UpdateBehavior': 'LOG',
            'DeleteBehavior': 'LOG'
        }
    )
    print(f"Crawler '{crawler_name}' created successfully.")
except glue.exceptions.AlreadyExistsException:
    print(f"Crawler '{crawler_name}' already exists.")
except Exception as e:
    print(f"Error creating crawler '{crawler_name}': {e}")

Crawler 'crawler_aggression_point' created successfully.


In [84]:
# 크롤러 동작
response = glue.start_crawler(Name=crawler_name)
print(f"Crawler '{crawler_name}' started successfully.")

Crawler 'crawler_aggression_point' started successfully.


## 공격성 교범 가중평균(정규화전)

In [ ]:
import awswrangler as wr
df = wr.athena.read_sql_query(
    """
    select a.school_code, a.school_name, b.school_grade, b.school_num, b.class_name, c.psy_name, c.psy_code, c.close_yn, f.student_num, d.target_code, e.user_testing_no, e.question_no, e.student_code 
    from school_info a
    inner join school_class b on a.school_code = b.school_code
    inner join psy_class c on b.class_code = c.class_code
    inner join psy_target d on c.class_code = d.class_code and c.psy_code = d.psy_code
    inner join at_user_testing_paper_pn e on d.user_testing_no = e.user_testing_no
    inner join school_student f on d.target_code = f.student_code and d.class_code = f.class_code
    where c.close_yn = 'Y' and e.question_no in ('4','5','6','7','14')
    """
    , database="schoolfriends")

# 친사회성 규범 점수

In [81]:
import awswrangler as wr
query_df = wr.athena.read_sql_query(
    """
    select a.school_code, a.school_name, b.school_grade, b.school_num, b.class_name, c.psy_name, c.psy_code, c.close_yn, f.student_num, d.target_code, e.user_testing_no, e.question_no, e.student_code 
    from school_info a
    inner join school_class b on a.school_code = b.school_code
    inner join psy_class c on b.class_code = c.class_code
    inner join psy_target d on c.class_code = d.class_code and c.psy_code = d.psy_code
    inner join at_user_testing_paper_pn e on d.user_testing_no = e.user_testing_no
    inner join school_student f on d.target_code = f.student_code and d.class_code = f.class_code
    where c.close_yn = 'Y' and e.question_no in (8,9,14)
    """
    , database="schoolfriends"
)

In [94]:
query_df

,school_code,school_name,school_grade,school_num,class_name,psy_name,psy_code,close_yn,student_num,target_code,user_testing_no,question_no,student_code
0,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",8,AC000120256042D2E004
1,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",9,AC000120256042D2E013
2,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",9,AC000120256042D2E014
3,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",14,AC000120256042D2E001
4,AC0001,학지초등학교,<NA>,<NA>,우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,<NA>,AC000120256042D2E020,"""557b6318ba464b62be79""",14,AC000120256042D2E004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9790,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,8,TC046120256076567022
9791,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,9,TC046120256076567022
9792,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567012
9793,TC0461,목포부주초등학교,<NA>,<NA>,"""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,<NA>,TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567021


In [95]:
# 지명수 구하는 함수
def count_point(df, student_list, item_no) :
    import pandas as pd
    if df['psy_code'].nunique() > 1 :
        raise ValueError('psy_code 컬럼에 여러 값이 존재합니다.')
    else :
        temp_df = df[df['question_no'].isin(item_no)] # 해당 문항 필터링
        count_list = []
        for student in student_list :
            if len(temp_df[temp_df['student_code']==student]) >= 1 :
                freq = len(temp_df[temp_df['student_code']==student])
            elif len(temp_df[temp_df['student_code']==student]) == 0 :
                freq = 0
            count_list.append(freq)
        count_point_df = pd.DataFrame({'student_code': student_list, 'num_point': count_list})
        return count_point_df, df['psy_code'].iloc[0]

# 학생 리스트 구하는 함수
def student_list(df) :
    student_list = df['target_code'].unique().tolist()
    return student_list, df['psy_code'].iloc[0]

In [85]:
#df = query_df[query_df['psy_code'].str.contains('P202507183B0A')]
#print(df)
df = query_df

In [103]:
#! 뭔가 이상해서 추출해 엑셀로 확인
import pandas as pd
df.to_excel(f'scoring_etl_check_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.xlsx', index=False)

In [176]:
student_list_obj, psy_code = student_list(df)
print(student_list_obj, psy_code)
print(len(student_list_obj))

['AC000120256042D2E020', 'AC000120256042D2E019', 'AC000120256042D2E018', 'AC000120256042D2E017', 'AC000120256042D2E016', 'AC000120256042D2E015', 'AC000120256042D2E014', 'AC000120256042D2E013', 'AC000120256042D2E012', 'AC000120256042D2E011', 'AC000120256042D2E010', 'AC000120256042D2E009', 'AC000120256042D2E008', 'AC000120256042D2E007', 'AC000120256042D2E006', 'AC000120256042D2E005', 'AC000120256042D2E004', 'AC000120256042D2E003', 'AC000120256042D2E002', 'AC000120256042D2E001', 'AC000120256016589025', 'AC000120256016589024', 'AC000120256016589023', 'AC000120256016589022', 'AC000120256016589021', 'AC000120256016589020', 'AC000120256016589019', 'AC000120256016589018', 'AC000120256016589017', 'AC000120256016589016', 'AC000120256016589015', 'AC000120256016589014', 'AC000120256016589013', 'AC000120256016589012', 'AC000120256016589011', 'AC000120256016589010', 'AC000120256016589009', 'AC000120256016589008', 'AC000120256016589007', 'AC000120256016589006', 'AC000120256016589005', 'AC000120256016

In [177]:
prosociality_point_df, psy_code = count_point(df, student_list_obj, ['8','9'])
n14_point_df, psy_code = count_point(df, student_list_obj, ['14'])
print(prosociality_point_df, psy_code)
print(n14_point_df, psy_code)

ValueError: psy_code 컬럼에 여러 값이 존재합니다.

In [96]:
def prosociality_norms(df, celling_parameter=249) :
    import pandas as pd
    student_list_obj, psy_code = student_list(df)
    prosociality_point_df, psy_code = count_point(df, student_list_obj, [8,9])
    n14_point_df, psy_code = count_point(df, student_list_obj, [14])

    prosociality_norms_df = pd.merge(prosociality_point_df, n14_point_df, on='student_code', suffixes=('_prosociality', '_14no'))
    prosociality_norms_df['prosociality*14no_num_point'] = prosociality_norms_df['num_point_prosociality'] * prosociality_norms_df['num_point_14no']
    prosociality_norms_df.sort_values(by='prosociality*14no_num_point', ascending=False, inplace=True)

    top5_prosociality_norms_df = prosociality_norms_df.head(5)
    top5_mean = top5_prosociality_norms_df['prosociality*14no_num_point'].mean()
    total_mean = prosociality_norms_df['prosociality*14no_num_point'].mean()
    weighted_mean = 0.4 * (top5_mean) + 0.6 * (total_mean)
    weighted_mean = 4 * ((min(max(weighted_mean, 0), celling_parameter)-0)/celling_parameter)+1
    result_df = pd.DataFrame({'psy_code': [psy_code], 'top5_mean': [top5_mean], 'total_mean': [total_mean], 'weighted_mean': [weighted_mean]})
    result_df['reg_date'] = pd.Timestamp.now()
    result_df['purpose'] = f'"purpose=calibration;celling_parameter={celling_parameter};author=Yumin"'
    return result_df

In [179]:
result_df = prosociality_norms(df)
print(result_df)

ValueError: psy_code 컬럼에 여러 값이 존재합니다.

In [180]:
df

,school_code,school_name,school_grade,school_num,class_name,psy_name,psy_code,close_yn,student_num,target_code,user_testing_no,question_no,student_code
0,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",8,AC000120256042D2E004
1,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",9,AC000120256042D2E013
2,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",9,AC000120256042D2E014
3,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",14,AC000120256042D2E001
4,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",14,AC000120256042D2E004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5062,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,8,TC046120256076567022
5063,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,9,TC046120256076567022
5064,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567012
5065,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567021


In [97]:
def search_celling_prosociality_norms(df, celling_parameter) :
    import pandas as pd
    student_list_obj, psy_code = student_list(df)
    prosociality_point_df, psy_code = count_point(df, student_list_obj, [8,9])
    n14_point_df, psy_code = count_point(df, student_list_obj, [14])

    prosociality_norms_df = pd.merge(prosociality_point_df, n14_point_df, on='student_code', suffixes=('_prosociality', '_14no'))
    prosociality_norms_df['prosociality*14no_num_point'] = prosociality_norms_df['num_point_prosociality'] * prosociality_norms_df['num_point_14no']
    prosociality_norms_df.sort_values(by='prosociality*14no_num_point', ascending=False, inplace=True)

    top5_prosociality_norms_df = prosociality_norms_df.head(5)
    top5_mean = top5_prosociality_norms_df['prosociality*14no_num_point'].mean()
    total_mean = prosociality_norms_df['prosociality*14no_num_point'].mean()
    weighted_mean = 0.4 * (top5_mean) + 0.6 * (total_mean)
    weighted_mean_Normalization = 4 * ((min(max(weighted_mean, 0), celling_parameter)-0)/celling_parameter)+1
    result_df = pd.DataFrame({'psy_code': [psy_code], 'top5_mean': [top5_mean], 'total_mean': [total_mean], 'weighted_mean': [weighted_mean], 'weighted_mean_Normalization': [weighted_mean_Normalization]})
    result_df['reg_date'] = pd.Timestamp.now()
    result_df['purpose'] = f'"purpose=calibration;celling_parameter={celling_parameter};author=Yumin"'
    return result_df

In [98]:
import pandas as pd

psy_code_grouped = df.groupby('psy_code')
result_df = pd.DataFrame()
for parameter in range(67, 250) :
    print(f'celling_parameter: {parameter}')
    for psy_code, group in psy_code_grouped :
        result_df = pd.concat([result_df, search_celling_prosociality_norms(group, celling_parameter=parameter)])

celling_parameter: 67
celling_parameter: 68
celling_parameter: 69
celling_parameter: 70
celling_parameter: 71
celling_parameter: 72
celling_parameter: 73
celling_parameter: 74
celling_parameter: 75
celling_parameter: 76
celling_parameter: 77
celling_parameter: 78
celling_parameter: 79
celling_parameter: 80
celling_parameter: 81
celling_parameter: 82
celling_parameter: 83
celling_parameter: 84
celling_parameter: 85
celling_parameter: 86
celling_parameter: 87
celling_parameter: 88
celling_parameter: 89
celling_parameter: 90
celling_parameter: 91
celling_parameter: 92
celling_parameter: 93
celling_parameter: 94
celling_parameter: 95
celling_parameter: 96
celling_parameter: 97
celling_parameter: 98
celling_parameter: 99
celling_parameter: 100
celling_parameter: 101
celling_parameter: 102
celling_parameter: 103
celling_parameter: 104
celling_parameter: 105
celling_parameter: 106
celling_parameter: 107
celling_parameter: 108
celling_parameter: 109
celling_parameter: 110
celling_parameter: 11

In [99]:
print(result_df)

         psy_code  top5_mean  total_mean  weighted_mean  \
0   P202507041AFF       8.80    8.800000       8.800000   
0   P202507046A44       3.60    3.600000       3.600000   
0   P202507075FAD       2.50    2.500000       2.500000   
0   P202507086502       3.25    3.250000       3.250000   
0   P20250708740B      43.00   20.000000      29.200000   
..            ...        ...         ...            ...   
0   P20250926990C       6.40    2.866667       4.280000   
0   P20250926E621      88.80   45.200000      62.640000   
0   P20250926F99B       8.00    3.533333       5.320000   
0   P202509297849     164.80   69.304348     107.502609   
0   P20250929B2C3      58.40   30.476190      41.645714   

    weighted_mean_Normalization                   reg_date  \
0                      1.525373 2025-10-01 15:29:57.500020   
0                      1.214925 2025-10-01 15:29:57.507314   
0                      1.149254 2025-10-01 15:29:57.513843   
0                      1.194030 2025-10-01 

In [184]:
result_df.to_csv(f'prosociality_norms_celling_parameter_search_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.csv', index=False)

In [100]:
# 결과를 S3에 저장
wr.s3.to_csv(result_df, f's3://schoolfriends-bym/result/prosociality_norms/prosociality_norms_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.csv', index=False)

{'paths': ['s3://schoolfriends-bym/result/prosociality_norms/prosociality_norms_20251001_153225.csv'],
 'partitions_values': {}}

In [131]:
# 친사회성 지명 데이터 산출
df

,school_code,school_name,school_grade,school_num,class_name,psy_name,psy_code,close_yn,student_num,target_code,user_testing_no,question_no,student_code
0,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",8,AC000120256042D2E004
1,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",9,AC000120256042D2E013
2,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",9,AC000120256042D2E014
3,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",14,AC000120256042D2E001
4,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",14,AC000120256042D2E004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5062,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,8,TC046120256076567022
5063,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,9,TC046120256076567022
5064,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567012
5065,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567021


In [101]:
# 지명수 구하는 함수
def count_point(df, student_list, item_no) :
    import pandas as pd
    if df['psy_code'].nunique() > 1 :
        raise ValueError('psy_code 컬럼에 여러 값이 존재합니다.')
    else :
        temp_df = df[df['question_no'].isin(item_no)] # 해당 문항 필터링
        count_list = []
        for student in student_list :
            if len(temp_df[temp_df['student_code']==student]) >= 1 :
                freq = len(temp_df[temp_df['student_code']==student])
            elif len(temp_df[temp_df['student_code']==student]) == 0 :
                freq = 0
            count_list.append(freq)
        count_point_df = pd.DataFrame({'student_code': student_list, 'num_point': count_list})
        return count_point_df, df['psy_code'].iloc[0]
    
# 학생 리스트 구하는 함수
def student_list(df) :
    student_list = df['target_code'].unique().tolist()
    return student_list, df['psy_code'].iloc[0]

In [102]:
import pandas as pd

psy_code_grouped = df.groupby('psy_code')
result_df = pd.DataFrame()
for psy_code, group in psy_code_grouped :
    student_list_obj, psy_code = student_list(group)
    prosociality_point_df, psy_code = count_point(group, student_list_obj, [8,9])
    n14_point_df, psy_code = count_point(group, student_list_obj, [14])
    merged_df = pd.merge(prosociality_point_df, n14_point_df, on='student_code', suffixes=('_prosociality', '_14no'))
    merged_df['prosociality*14no_num_point'] = merged_df['num_point_prosociality'] * merged_df['num_point_14no']
    merged_df['psy_code'] = psy_code
    print(merged_df)
    result_df = pd.concat([result_df, merged_df])

              student_code  num_point_prosociality  num_point_14no  \
0  M1000000120253017C9D005                       4               2   
1  M1000000120253017C9D004                       5               4   
2  M1000000120253017C9D003                       5               3   
3  M1000000120253017C9D002                       1               0   
4  M1000000120253017C9D001                       1               1   

   prosociality*14no_num_point       psy_code  
0                            8  P202507041AFF  
1                           20  P202507041AFF  
2                           15  P202507041AFF  
3                            0  P202507041AFF  
4                            1  P202507041AFF  
              student_code  num_point_prosociality  num_point_14no  \
0  M1000000120253017C9D005                       2               2   
1  M1000000120253017C9D004                       5               2   
2  M1000000120253017C9D003                       2               2   
3  M1000000

In [103]:
print(result_df)

               student_code  num_point_prosociality  num_point_14no  \
0   M1000000120253017C9D005                       4               2   
1   M1000000120253017C9D004                       5               4   
2   M1000000120253017C9D003                       5               3   
3   M1000000120253017C9D002                       1               0   
4   M1000000120253017C9D001                       1               1   
..                      ...                     ...             ...   
16     DC03142025602FCDA005                       7               2   
17     DC03142025602FCDA004                       9               8   
18     DC03142025602FCDA003                       7               7   
19     DC03142025602FCDA002                       5               3   
20     DC03142025602FCDA001                       9               2   

    prosociality*14no_num_point       psy_code  
0                             8  P202507041AFF  
1                            20  P202507041AFF  


In [104]:
# s3에 업로드
wr.s3.to_csv(result_df, f's3://schoolfriends-bym/result/prosociality_point/prosociality_point_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.csv', index=False)

{'paths': ['s3://schoolfriends-bym/result/prosociality_point/prosociality_point_20251001_153226.csv'],
 'partitions_values': {}}

In [32]:
# s3_folder_macro 소스에서 크롤러 생성해서 테이블화
# 특정 크롤러 생성 및 실행
import boto3
# s3 폴더 별로 크롤러 만들어서 아테네 테이블 만들때
glue = boto3.client("glue", region_name="ap-northeast-2")

# 공통 파라미터
role_name = 'arn:aws:iam::932744610695:role/service-role/AWSGlueServiceRole-Schoolfriends'
database_name = 'schoolfriends'
bucket_name = 'schoolfriends-bym'

crawler_name = "crawler_prosociality_norms"
s3_target_path = f"s3://schoolfriends-bym/result/prosociality_norms/"

try:
    response = glue.create_crawler(
        Name=crawler_name,
        Role=role_name,
        DatabaseName=database_name,
        Targets={'S3Targets': [{'Path': s3_target_path}]},
        SchemaChangePolicy={
            'UpdateBehavior': 'LOG',
            'DeleteBehavior': 'LOG'
        }
    )
    print(f"Crawler '{crawler_name}' created successfully.")
except glue.exceptions.AlreadyExistsException:
    print(f"Crawler '{crawler_name}' already exists.")
except Exception as e:
    print(f"Error creating crawler '{crawler_name}': {e}")

Crawler 'crawler_prosociality_norms' created successfully.


In [29]:
# 크롤러 동작
response = glue.start_crawler(Name=crawler_name)
print(f"Crawler '{crawler_name}' started successfully.")

NameError: name 'glue' is not defined